In [3]:
%tensorflow_version 1.x

TensorFlow is already loaded. Please restart the runtime to change versions.


### Importer les packages nécessaires.

In [0]:
import numpy as np
import keras.backend as K
from keras.layers import Input, Lambda
from keras.models import Model
from keras.optimizers import Adam
from train.build_train import preprocess_true_boxes, build_model_net, yolo_loss
from train.train_util import get_random_data

### Copier la Dataset du drone et les poids pré-entraînés sur MS COCO depuis le drive.

In [0]:
# Copier la dataset depuis le drive.
!cp drive/"My Drive"/amodelzzz/drone_dataset.zip ./
# Copier les poids entrainés sur MS COCO depuis le drive
!cp drive/"My Drive"/amodelzzz/model_weights_only.h5 .
# Décompresser la dataset et supprimer le fichier zip.
!unzip drone_dataset.zip
!rm drone_dataset.zip

### Définir une fonction qui lit les coordonnées des anchors boxes depuis un txt.

In [0]:
def get_anchors(anchors_path):
    with open(anchors_path) as f:
        anchors = f.readline()
    anchors = [float(x) for x in anchors.split(',')]
    return np.array(anchors).reshape(-1, 2)

### Définir une fonction qui prépare créée et prépare le modèle pour l'entraînement.

In [0]:
def create_model(input_shape, anchors, num_classes, weights_path='model_weights_only.h5'):
    K.clear_session() 
    image_input = Input(shape=(None, None, 3))
    h, w = input_shape
    num_anchors = len(anchors)
    """
     Création du placeholder 'y_true' qui va contenir les 3 outputs du model.
     Les 3 outputs du model seront de taille 52, 26 et 13.
     Les 3 outputs du model seront respectivement au niveau des layers: 82, 94 et 106.
   """
    y_true = [Input(shape=(h//{0:32, 1:16, 2:8}[l], w//{0:32, 1:16, 2:8}[l], \
        num_anchors//3, num_classes+5)) for l in range(3)]
    # Création de l'architecture du model.
    model_body = build_model_net()
    print('On a créé le model ayant {} anchors et {} classes d\'objets.'.format(num_anchors, num_classes))
    
    # Charger les weights. (pre-trained weights)
    model_body.load_weights(weights_path, by_name=True, skip_mismatch=True)
    print('On a chargé les Weights initiaux. {}.'.format(weights_path))
    
    
    # Bloquer toute les couches du modèle et laisser les 3 couches de sorties.
    num = len(model_body.layers)-3
    for i in range(num):
      model_body.layers[i].trainable = False
    # Configurer la fonction loss comme layer Lambda sous nom: yolo_loss.
    model_loss = Lambda(yolo_loss, output_shape=(1,), name='yolo_loss',
        arguments={'anchors': anchors, 'num_classes': num_classes, 'ignore_thresh': .5})(
        [*model_body.output, *y_true])
    model = Model([model_body.input, *y_true], model_loss)
 
    return model

### Définir la fonction data_generator qui pré-traîte les données pour le training.
Ainsi que distordre les images via la fonction `get_random_data` avec les paramètres :
jitter=0.3, hue=0.1 et saturation=1.5

In [0]:
def data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    i = 0
    while True:
        image_data = []
        box_data = []
        for b in range(batch_size):
            if i==0:
                np.random.shuffle(annotation_lines)
            image, box = get_random_data('drone_dataset_yolo/dataset_txt/'+annotation_lines[i], input_shape, random=True)
            image_data.append(image)
            box_data.append(box)
            i = (i+1) % n
        image_data = np.array(image_data)
        box_data = np.array(box_data)
        y_true = preprocess_true_boxes(box_data, input_shape, anchors, num_classes)
        yield [image_data, *y_true], np.zeros(batch_size)
 
def data_generator_wrapper(annotation_lines, batch_size, input_shape, anchors, num_classes):
    n = len(annotation_lines)
    if n==0 or batch_size<=0: return None
    return data_generator(annotation_lines, batch_size, input_shape, anchors, num_classes)

### Définir une fonction qui lit les labels(catégories ou classes).

In [0]:
def read_labels(f):
  labels_file = open(f,'r')
  labels_list = labels_file.read().split('\n')
  labels_list.pop()
  return labels_list

### sauvegarder les liens vers le fichier d'annotations, d'anchors et des classes. Ainsi que l'input shape qui est la taille de l'image d'entrée imposée par le modèle.



In [0]:
annotation_path = 'annots.txt' # accès aux annotations.
# lire les labels depuis un fichier texte. (MSCOCO labels)
labels = read_labels('labels.txt')
anchors_filename = 'anchors.txt' # path pour lire anchors.
# on sauvgarde le nombre de classes (qui est 80).
num_classes = len(labels)
# lisons les anchors depuis le fichier des anchors.
anchors = get_anchors(anchors_filename)
input_shape=(416,416)

### Créer un modèle au préalable chargé des poids pré-entraînés.
Aussi, bloquer toutes les couches sauf les 3 couches de sorties qui sont déstinées à s'entraîner.

In [11]:
model = create_model(input_shape, anchors, num_classes, weights_path='model_weights_only.h5')


On a créé le model ayant 9 anchors et 80 classes d'objets.
On a chargé les Weights initiaux. model_weights_only.h5.


### Détermination du nombre du lot de training/lot de validation.

In [0]:
val_split = 0.2 # lot de validation a pour portion 20%.
with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(33)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines)*val_split)
num_train = len(lines) - num_val

### Lancer le training en utilisant l'optimisateur Adam de learning_rate=0.001.

In [0]:
# Utiliser la fonction loss qu'on a définit sous le nom : yolo_loss.
# Utiliser Adam comme optimisateur avec un learning_rate alpha = .001
model.compile(optimizer=Adam(lr=1e-3), loss={'yolo_loss': lambda y_true, y_pred: y_pred})
 
batch_size = 32
print('Faire le training du model sur {} échantillions, validation sur {} échantillions, avec un batch size {}.'.format(num_train, num_val, batch_size))
model.fit_generator(
    # Fournir le data_generator.
    data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes),
        # Fournir la longueur de chaque epoch.
        steps_per_epoch=max(1, num_train//batch_size),
        validation_data=data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes),
        # FOurnir la longueur de chaque validation.
        validation_steps=max(1, num_val//batch_size),
        # on va utiliser 750 epochs.
        epochs=750,
        # En commençant par l'epoch 0.
        initial_epoch=0)
# Sauvegarder les weights sous fichier .h5 (format de keras).
model.save_weights('trained_weights_stage_1.h5')